In [1]:
from utils.nnn_tools import agent2, company_ids,cid_qe_tool, sql_query_engine
from llama_index.core import ServiceContext
from llama_index.core import Settings

gpt-3.5-turbo
gpt-4o


loading from disk
0
loading from disk


In [2]:
from thefuzz import  fuzz

In [9]:
Settings.llm.model

'gpt-3.5-turbo'

In [3]:
sql_query_engine.query("koji korisnik ima najviše pretplatnika koji ugovor ističe za manje od 3 mjeseca")

Response(response='Korisnik s OIB-om 79174502895 ima najviše pretplatnika čiji ugovor ističe za manje od 3 mjeseca, ukupno 18 pretplatnika.', source_nodes=[NodeWithScore(node=TextNode(id_='741640cf-83d5-4c2b-ba5e-c6f061193093', embedding=None, metadata={'sql_query': 'SELECT oib, COUNT(subscriber_id) AS broj_pretplatnika\nFROM mobile_customer_base\nWHERE Fees_to_charge < 3\nGROUP BY oib\nORDER BY broj_pretplatnika DESC\nLIMIT 1;', 'result': [(79174502895, 18)], 'col_keys': ['oib', 'broj_pretplatnika']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, text='[(79174502895, 18)]', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'741640cf-83d5-4c2b-ba5e-c6f061193093': {'sql_query': 'SELECT oib, COUNT(subscriber_id) AS broj_pretplatnika\nFROM mobile_

In [3]:
from llama_index.core import PromptTemplate

In [9]:
PromptTemplate(prompt_change)

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['dialect', 'schema', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='\n            Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.\n\n            Never query for all the columns from a specific table, only ask for a few relevant columns given the question.\n\n            Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:\n\n            Question: Question here\n            SQLQuery

In [3]:
sql_query_engine.get_prompts()

{'response_synthesis_prompt': PromptTemplate(metadata={'prompt_type': <PromptType.SQL_RESPONSE_SYNTHESIS_V2: 'sql_response_synthesis_v2'>}, template_vars=['query_str', 'sql_query', 'context_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Given an input question, synthesize a response from the query results.\nQuery: {query_str}\nSQL: {sql_query}\nSQL Response: {context_str}\nResponse: '),
 'sql_retriever:text_to_sql_prompt': '\n            Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.\n\n            Never query for all the columns from a specific table, only ask for a few relevant columns given the question.\n\n            Pay attention to use only the column names that you can see in the schema description. Be careful to not query fo

In [14]:
sql_query_engine._refine_synthesis_prompt()

PromptTemplate(metadata={'prompt_type': <PromptType.REFINE: 'refine'>}, template_vars=['query_str', 'existing_answer', 'context_msg'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template="The original query is as follows: {query_str}\nWe have provided an existing answer: {existing_answer}\nWe have the opportunity to refine the existing answer (only if needed) with some more context below.\n------------\n{context_msg}\n------------\nGiven the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.\nRefined Answer: ")

In [8]:
prompt_change = """
            Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

            Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

            Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

            Question: Question here
            SQLQuery: SQL Query to run
            SQLResult: Result of the SQLQuery
            Answer: Final answer here

            Only use tables listed below.
            {schema}

            examples:
            Question: what customer has the most subscribers
            SQLQuery: SELECT oib, COUNT(subscriber_id) AS num_subscribers\nFROM mobile_customer_base\nGROUP BY oib\nORDER BY num_subscribers DESC\nLIMIT 1;
            Question: customer with the most subscribers with contract expiry less than 3
            SQLQuery: SELECT oib, COUNT(subscriber_id) AS num_subscribers\nFROM mobile_customer_base\nWHERE Fees_to_charge < 3\nGROUP BY oib\nORDER BY num_subscribers DESC\nLIMIT 1;

            Question: {query_str}
            SQLQuery: 
    """

#n3p.sql_qe_template["query_tool_custom_prompt"]
#sql_query_engine.update_prompts(prompt_change)

In [10]:
print(sql_query_engine.get_prompts()['sql_retriever:text_to_sql_prompt'].template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 


In [11]:
agent2.chat("pojavljuje li se kompaniji u vijestima")

Added user message to memory: pojavljuje li se kompaniji u vijestima
=== LLM Response ===
Da, tvrtka KOFEIN d.o.o. se pojavljuje u vijestima. Evo nekoliko informacija o prisustvu tvrtke u vijestima:

1. Kreativna agencija Kofein u manje od godine dana zaposlila je 17 osoba i ostvarila značajan poslovni uspjeh, uspostavljajući kvalitetan kreativni odjel koji dobiva pitcheve i natječaje (netokracija.com).
2. KOFEIN d.o.o. je u 2022. ostvario neto rezultat poslovanja od 62.549,00 € i povećao broj zaposlenih na 29 osoba (poslovna.hr).

Ove informacije pružaju uvid u aktivnosti i uspjehe tvrtke KOFEIN d.o.o. u poslovanju i kreativnom sektoru.


AgentChatResponse(response='Da, tvrtka KOFEIN d.o.o. se pojavljuje u vijestima. Evo nekoliko informacija o prisustvu tvrtke u vijestima:\n\n1. Kreativna agencija Kofein u manje od godine dana zaposlila je 17 osoba i ostvarila značajan poslovni uspjeh, uspostavljajući kvalitetan kreativni odjel koji dobiva pitcheve i natječaje (netokracija.com).\n2. KOFEIN d.o.o. je u 2022. ostvario neto rezultat poslovanja od 62.549,00 € i povećao broj zaposlenih na 29 osoba (poslovna.hr).\n\nOve informacije pružaju uvid u aktivnosti i uspjehe tvrtke KOFEIN d.o.o. u poslovanju i kreativnom sektoru.', sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)

In [62]:
# Check the similarity score
name = "Matko Pavisicii"
full_name = "Marko Pavisic"
print(f"Similarity score: {fuzz.ratio(name, full_name)}")

Similarity score: 86


In [3]:
import pandas as pd

In [4]:
subscriber_level_df_final = pd.read_csv("data/synthetic_data_cb_w_oib.csv")

In [13]:
subscriber_level_df_final.query("oib==60770198608").groupby(["tariff_model"])[["Voice_usage","Roaming_usage"]].mean().reset_index()

,tariff_model,Voice_usage,Roaming_usage
0,l,3064.500000,241.000000
1,m,2133.400000,221.500000
2,s,3515.333333,193.666667
3,xl,2838.600000,254.200000


In [66]:
input2 ="PRVO"

ratio_score = lambda name,full_name : fuzz.ratio(name, full_name)

name2oib = [{"oib": i[0], "naziv": i[1], "score":ratio_score(input2,i[1])} for i in zip(company_ids["OIB"],company_ids["NAZIV"]) if input2.lower() in i[1].lower() ]

In [67]:
name2oib

[]

In [21]:
cid_qe_tool.call("HEP")

ToolOutput(content="{\n  'oib': '98686076589',\n  'naziv': 'EUVITIS'\n}", tool_name='company_name_and_id_finder', raw_input={'input': 'HEP'}, raw_output=Response(response="{\n  'oib': '98686076589',\n  'naziv': 'EUVITIS'\n}", source_nodes=[NodeWithScore(node=TextNode(id_='2daaf688-ccdd-42b4-94bd-bd66b42b893b', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3d7040ac-7bb8-406f-a1da-3279e7613de2', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='727472967328cc7534d7f750bca1116ac8fabb94e00f156eddd116f51f8870ef')}, text='oib (vat_id): 98686076589, naziv: EUVITIS, obrt za stručne, znanstvene i obrazovne usluge, vl. Edi Maletić, Zagreb, Jablanska 70', mimetype='text/plain', start_char_idx=0, end_char_idx=128, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.7663371855099151), NodeWithScore(node=TextNode(

In [ ]:
def oib2name_func(name:str):
    """
    input is string
    Finds company name based on name input.
    Use this tool when user inputs oib and you need company name
    output is dictionary with oib and name of the company
    """
    oib2name_ = oib2name

    return str(oib2name_[oib])


tool_oib2name = FunctionTool.from_defaults(oib2name_func, 
name="oib2name",
description= """
    Finds company name based on oib input.
    Use this tool when user inputs oib and you need company name
    output is dictionary with oib and name of the company
    """,
return_direct=True)


In [2]:
import pandas as pd

In [3]:
subscriber_level_df_final = pd.read_csv("data/synthetic_data_cb_w_oib.csv")

from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt4o")

from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:")
subscriber_level_df_final.to_sql(
  "mobile_customer_base",
  engine
)

tables = ["mobile_customer_base"]

from llama_index.core import SQLDatabase

sql_database = SQLDatabase(
    engine=engine,
    include_tables=tables,
    
    
)

from llama_index.core.indices.struct_store.sql_query import NLSQLTableQueryEngine

examples = [
        {"input":"what customer has the most subscribers","sql_query":"SELECT oib, COUNT(subscriber_id) AS num_subscribers\nFROM mobile_customer_base\nGROUP BY oib\nORDER BY num_subscribers DESC\nLIMIT 1;"},
        {"input":"customer with the most subscribers with contract expiry less than 3",
        "sql_query":"SELECT oib, COUNT(subscriber_id) AS num_subscribers\nFROM mobile_customer_base\nWHERE Fees_to_charge < 3\nGROUP BY oib\nORDER BY num_subscribers DESC\nLIMIT 1;"},
    ]

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=tables,
    examples=examples,

    
    
)


DESCRIPTION = """\
This tool should be used to answer question related to the mobile by translating a natural language query into a SQL query with access to tables:
first try to use queires provided in examples
'mobile_customer_base' - primary filter column is oib, id_columns are - oib, customer_id, subscriber_id, ARPA_2024 is arpa column, fees_to_charge is contract expiry column


"""


from llama_index.core.tools.query_engine import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql-query",
    description=DESCRIPTION,

    
)


In [142]:
print(sql_query_engine.get_prompts()['sql_retriever:text_to_sql_prompt'].template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 


In [4]:
prompt_change = """
Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

examples:
Question: what customer has the most subscribers
SQLQuery: SELECT oib, COUNT(subscriber_id) AS num_subscribers\nFROM mobile_customer_base\nGROUP BY oib\nORDER BY num_subscribers DESC\nLIMIT 1;
Question: customer with the most subscribers with contract expiry less than 3
SQLQuery: SELECT oib, COUNT(subscriber_id) AS num_subscribers\nFROM mobile_customer_base\nWHERE Fees_to_charge < 3\nGROUP BY oib\nORDER BY num_subscribers DESC\nLIMIT 1;

Question: {query_str}
SQLQuery: 
"""
sql_query_engine.get_prompts()['sql_retriever:text_to_sql_prompt'].template=prompt_change

In [21]:
subscriber_level_df_final.query("Fees_to_charge<6").groupby(["oib"])["customer_id"].count().reset_index().sort_values("customer_id")

,oib,customer_id
216,63914995885,1
148,43420386536,1
293,87096588160,1
163,49129895772,1
239,71685046796,1
...,...,...
172,50538528899,28
39,9661526637,30
149,43496259355,32
306,91467117650,32


In [141]:
sql_query_engine.update_prompts(examples)

In [6]:
sql_query_engine.query("what oib has the most subscribers with months till contract expiration less than 12").response

'The OIB with the most subscribers who have less than 12 months till contract expiration is 43496259355, with a total of 42 subscribers.'

In [ ]:
sql_query_engine.query("what oib has the most subscribers with months till contract expiration less than 12").response

In [127]:
sql_tool.call("SELECT customer_id, COUNT(subscriber_id) AS num_subscribers\\nFROM mobile_customer_base\\nWHERE Duration_type < 3\\nGROUP BY customer_id\\nORDER BY num_subscribers DESC\\nLIMIT 1;")

ToolOutput(content='The SQL query provided is invalid. Please check the syntax and try again.', tool_name='sql-query', raw_input={'input': 'SELECT customer_id, COUNT(subscriber_id) AS num_subscribers\\nFROM mobile_customer_base\\nWHERE Duration_type < 3\\nGROUP BY customer_id\\nORDER BY num_subscribers DESC\\nLIMIT 1;'}, raw_output=Response(response='The SQL query provided is invalid. Please check the syntax and try again.', source_nodes=[NodeWithScore(node=TextNode(id_='ba0e3636-af6f-4f5d-8409-fa6e41860207', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Error: Statement 'SELECT customer_id, COUNT(subscriber_id) AS num_subscribers\\nFROM mobile_customer_base\\nWHERE Duration_Type < 3\\nGROUP BY customer_id\\nORDER BY num_subscribers DESC\\nLIMIT 1;' is invalid SQL.", mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadat

In [116]:
from llama_index.core.indices.struct_store.sql_query import NLSQLTableQueryEngine, SQLDatabase
from llama_index.core.prompts import Prompt, PromptType

examples = [
        {"input":"what customer has the most subscribers","sql_query":"SELECT oib, COUNT(subscriber_id) AS num_subscribers\nFROM mobile_customer_base\nGROUP BY oib\nORDER BY num_subscribers DESC\nLIMIT 1;"},
        {"input":"customer with the most subscribers with contract expiry less than 3",
        "sql_query":"SELECT oib, COUNT(subscriber_id) AS num_subscribers\nFROM mobile_customer_base\nWHERE Fees_to_charge < 3\nGROUP BY oib\nORDER BY num_subscribers DESC\nLIMIT 1;"},
    ]

class ExampleDrivenNLSQLTableQueryEngine(NLSQLTableQueryEngine):
    def __init__(self, sql_database: SQLDatabase, examples: list, **kwargs):
        super().__init__(sql_database, **kwargs)
        self.examples = examples

    def _generate_sql_query(self, query: str) -> str:
        prompt_text = "Here are some examples of natural language queries and their corresponding SQL queries:\n\n"
        for example in self.examples:
            prompt_text += f"Q: {example['input']}\nA: {example['output']}\n\n"
        prompt_text += f"Now, generate a SQL query for the following question:\nQ: {query}\nA:"

        prompt = Prompt(
            template=prompt_text,
            prompt_type=PromptType.TEXT_TO_SQL
        )
        
        response = self._llm.generate(prompt)
        return response
# Initialize the custom query engine
query_engine = ExampleDrivenNLSQLTableQueryEngine(sql_database=sql_database, examples=examples)


In [123]:
query_engine.query("""customer with the most subscribers with contract expiration less than 3
Examples:
{examples}

""")

Response(response="I'm sorry, but there seems to be an error in the SQL query provided. Please double-check the syntax and try again.", source_nodes=[NodeWithScore(node=TextNode(id_='e2c5179c-589f-44e5-a56b-771eddf409cf', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Error: Statement 'SELECT customer_id, COUNT(subscriber_id) AS num_subscribers\\nFROM mobile_customer_base\\nWHERE Duration_Type < 3\\nGROUP BY customer_id\\nORDER BY num_subscribers DESC\\nLIMIT 1;' is invalid SQL.", mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'e2c5179c-589f-44e5-a56b-771eddf409cf': {}, 'sql_query': 'SELECT customer_id, COUNT(subscriber_id) AS num_subscribers\nFROM mobile_customer_base\nWHERE Duration_Type < 3\nGROUP BY customer_id\nORDER BY num_subscribers DESC\nLIMIT 1;'})